Fetching Webpages with the Requests Library To Demonstrate
===========================================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [287]:
from sqlalchemy import *
import datetime
import mediacloud

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [288]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
echo=True
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('sentenceCount',Integer),
)
metadata.create_all(engine) # and create the tables in the database

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [289]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, "sentenceCount") VALUES (:id, :keywords, :timestamp, :sentenceCount)'

In [290]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt)

'INSERT INTO queries (keywords, timestamp) VALUES (:keywords, :timestamp)'

In [291]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[3]

In [292]:
countThis = 'Trump'
mc = mediacloud.api.MediaCloud("b78d27ab136fdca2b4adac433487cc1c4aa0f837f5a0c49bd6ae135a22722486")
filter_set = [mc.publish_date_query(datetime.date(2015,8,1),datetime.date(2015,8,31)),'media_sets_id:1']
resultCount = mc.sentenceCount(countThis,solr_filter=filter_set)
print countThis + ': ' + str(resultCount['count'])
insert_stmt = queries.insert().values(keywords=countThis, sentenceCount=resultCount['count'])

Trump: 34800


In [293]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[1]

In [294]:
countThis = 'Obama'
mc = mediacloud.api.MediaCloud("b78d27ab136fdca2b4adac433487cc1c4aa0f837f5a0c49bd6ae135a22722486")
filter_set = [mc.publish_date_query(datetime.date(2015,8,1),datetime.date(2015,8,31)),'media_sets_id:1']
resultCount = mc.sentenceCount(countThis,solr_filter=filter_set)
print countThis + ': ' + str(resultCount['count'])
insert_stmt = queries.insert().values(keywords=countThis, sentenceCount=resultCount['count'])

Obama: 15235


In [295]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[2]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [296]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

(1, u'Trump', datetime.datetime(2015, 12, 14, 22, 17, 10, 416000), 34800)
(2, u'Obama', datetime.datetime(2015, 12, 14, 22, 17, 11, 308000), 15235)


In [297]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

(1, u'Trump', datetime.datetime(2015, 12, 14, 22, 17, 10, 416000), 34800)


## Adding Up

In [298]:
select_stmt = select([queries]).where(queries.c.sentenceCount)
#print select_stmt #debug
sum=0
for row in db_conn.execute(select_stmt):
    sum = sum + row.sentenceCount
print 'Sum is ' + str(sum)

sum is 50035


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [299]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [300]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function <lambda> at 0x05939B70>)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [301]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [302]:
query = Query(keywords="iguana")
query.keywords

'iguana'

In [303]:
my_session.add(query)
my_session.commit()
query.id

1

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [304]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana')>


In [305]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [306]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana')>
<Query(keywords='robot')>
<Query(keywords='puppy')>


In [307]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q

<Query(keywords='robot')>
